### 직방 서비스 원룸 데이터 수집
- 복잡한 데이터 수집 방법
- 동 이름 > 매물 정보(DataFrame)

- 절차
    - 동이름 > 위도, 경도
    - 위도, 경도 > geohash code(지역 범위)
    - geohash > 매물 아이디
    - 매물 아이디 > 매물 정보
    

In [10]:
import pandas as pd
import requests

In [ ]:
# 1. 동이름 > 위도, 경도

In [14]:
query = '망원동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={query}&serviceType=원룸'
response = requests.get(url)
datas = response.json()['items'][0]
lat, lng = datas['lat'], datas['lng']
lat, lng

(37.556785583496094, 126.9013442993164)

In [ ]:
# 2. 위도, 경도 > geohash code(지역범위)

In [18]:
import geohash2

In [19]:
# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydjx'

In [ ]:
# 3. gehash > 매무 아이디

In [44]:
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
response = requests.get(url)
datas = response.json()['items']
datas

[{'lat': 37.529612191172745, 'lng': 126.89760172581643, 'item_id': 30462357},
 {'lat': 37.52910125879532, 'lng': 126.89695233849588, 'item_id': 30524930},
 {'lat': 37.529599329339646, 'lng': 126.89760973265426, 'item_id': 30629925},
 {'lat': 37.529577694647344, 'lng': 126.8976003851259, 'item_id': 30681505},
 {'lat': 37.529103859419884, 'lng': 126.89130672348378, 'item_id': 30494154},
 {'lat': 37.52980853009123, 'lng': 126.8979681915614, 'item_id': 30351843},
 {'lat': 37.529785119606956, 'lng': 126.89793303011601, 'item_id': 30620037},
 {'lat': 37.52991196900484, 'lng': 126.90659912269808, 'item_id': 30218399},
 {'lat': 37.53106074704453, 'lng': 126.90697734846236, 'item_id': 30673258},
 {'lat': 37.53109143690658, 'lng': 126.90696335616244, 'item_id': 30673399},
 {'lat': 37.530792957427145, 'lng': 126.90695829523004, 'item_id': 30355099},
 {'lat': 37.53079157275767, 'lng': 126.90695312350115, 'item_id': 30444123},
 {'lat': 37.53128387548199, 'lng': 126.90695951038656, 'item_id': 304663

In [45]:
# item_ids = []
# for i in datas :
#     item_ids.append(i['item_id'])
# item_ids

ids = [i['item_id'] for i in datas]
ids

[30462357,
 30524930,
 30629925,
 30681505,
 30494154,
 30351843,
 30620037,
 30218399,
 30673258,
 30673399,
 30355099,
 30444123,
 30466344,
 30560609,
 30561299,
 30573310,
 30574604,
 30610378,
 30615350,
 30621726,
 30627111,
 30628305,
 30634455,
 30655847,
 30685231,
 30689836,
 30689856,
 30695080,
 30705142,
 30705883,
 30515891,
 30634391,
 30685631,
 30538698,
 30558005,
 30593189,
 30679856,
 30693696,
 30704747,
 30708132,
 30732018,
 30707152,
 30537418,
 30496424,
 30521128,
 30666644,
 30692066,
 30354996,
 30491596,
 30541220,
 30567670,
 30577800,
 30587646,
 30588047,
 30605540,
 30640328,
 30649984,
 30653524,
 30660847,
 30667810,
 30679820,
 30681674,
 30682215,
 30683248,
 30684594,
 30690110,
 30693868,
 30713425,
 30729962,
 30731143,
 30558890,
 30581225,
 30373907,
 30687897,
 30679700,
 30669838,
 30267901,
 30388530,
 30388587,
 30393190,
 30393301,
 30474061,
 30545029,
 30553458,
 30566381,
 30566391,
 30595775,
 30607679,
 30650735,
 30650763,
 30664217,

In [46]:
# 4. 매물 아이디 > 매물 정보

In [68]:
url = 'https://apis.zigbang.com/v2/items/list'
params = {
    'domain': "zigbang",
    "item_ids" : ids,
    'withCoalition': 'true',
}
response = requests.post(url, params)
response

<Response [200]>

In [69]:
pd.options.display.max_columns = 30

In [70]:
pd.options.display.max_rows

60

In [71]:
items = response.json()['items']
# columns = ['item_id', 'sales_type', 'deposit', 'size_m2', '전용면적']
item_df = pd.DataFrame(items, columns=columns)
item_df.tail(2)

,item_id,sales_title,deposit,rent,size_m2,floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
913,30455640,전세,28000,0,33.58,6,🔶🔶마포 저렴하고 전망 좋은 복층 쓰리룸 탑층🔶🔶,05,마포구 중동,빌라,서울시 마포구 중동,3,2022-02-03T12:00:40+09:00,False
914,30687168,월세,5000,60,41.36,3,🟥🟨🟩 깔끔한 분위기~! 넓은 신축형 복층 원룸,04,마포구 중동,빌라,서울시 마포구 중동,8,2022-02-18T14:00:36+09:00,False


In [72]:
item_df.columns

Index(['item_id', 'sales_title', 'deposit', 'rent', 'size_m2', 'floor',
       'title', 'room_type', 'address', 'service_type', 'address1',
       'manage_cost', 'reg_date', 'is_new'],
      dtype='object')

In [73]:
columns = ['item_id','sales_title', 'deposit', 'rent', 'size_m2','floor','title',
           'room_type', 'address','service_type', 'address1', 'manage_cost', 'reg_date', 'is_new']

In [74]:
result_df = item_df[columns]
result_df

,item_id,sales_title,deposit,rent,size_m2,floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
0,30462357,전세,8000,0,19.83,2,"융자없는 풀옵션전세원룸 입주즉시가능 청소,도배완료",01,영등포구 당산동4가,원룸,서울시 영등포구 당산동4가,13,2022-02-18T17:43:12+09:00,False
1,30524930,전세,7000,0,16.53,7,💕놓칠수없는가격💕파격EVENT💕조절가능💕즉입가능💕,01,영등포구 당산동4가,원룸,서울시 영등포구 당산동4가,13,2022-02-07T16:03:16+09:00,False
2,30629925,전세,7000,0,19.83,7,🧡더블역세권💥착한전세가💥밝은주변환경💥융자❌💛,01,영등포구 당산동4가,원룸,서울시 영등포구 당산동4가,12,2022-02-19T16:52:02+09:00,True
3,30681505,전세,7000,0,19.83,7,★실제매물입니다★ 영등포구청역 융자없는 깔끔한 원룸입니다.,01,영등포구 당산동4가,원룸,서울시 영등포구 당산동4가,13,2022-02-17T15:58:52+09:00,False
4,30494154,전세,6000,0,19.83,5,※양평역 원룸 전세※ #전세대출불가 #대로변 #가성비,01,영등포구 양평동3가,원룸,서울시 영등포구 양평동3가,12,2022-02-05T11:14:17+09:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
910,30653716,월세,4000,27,15.00,2,🐯성산동 신축급 풀옵션🐯,01,마포구 성산동,원룸,서울시 마포구 성산동,8,2022-02-19T12:13:37+09:00,False
911,30673351,월세,9000,20,19.83,1,🌿 테라스 있는 반전세 원룸 / 월세 가능 / 신축 🌿,01,마포구 성산동,원룸,서울시 마포구 성산동,8,2022-02-17T10:27:46+09:00,False
912,30629059,전세,26000,0,65.00,3,올리모델링한 전용 넓은 쓰리룸 전세,05,마포구 성산동,빌라,서울시 마포구 성산동,0,2022-02-14T14:50:13+09:00,False
913,30455640,전세,28000,0,33.58,6,🔶🔶마포 저렴하고 전망 좋은 복층 쓰리룸 탑층🔶🔶,05,마포구 중동,빌라,서울시 마포구 중동,3,2022-02-03T12:00:40+09:00,False


In [75]:
query

'망원동'

In [77]:
result_df = result_df[result_df['address'].str.contains(query)]
result_df =result_df.reset_index(drop=True)
result_df.head()

,item_id,sales_title,deposit,rent,size_m2,floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
0,30483493,전세,23000,0,18.99,6,"🦋합정역,신축1.5룸가성비짱,한강공원도보,보증보험OK",02,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-04T15:40:23+09:00,False
1,30608827,전세,23000,0,42.98,6,방1개 별도거실 주방 화장실 다용도실 분리형원룸 입니다,04,마포구 망원동,빌라,서울시 마포구 망원동,7,2022-02-12T16:27:27+09:00,False
2,30578522,월세,12000,10,12.21,3,💥LH 가능 깨끗한 방💥,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-15T18:54:14+09:00,False
3,30726074,월세,12000,10,13.22,3,🎶 [LH 가능!] 🎶 [깔끔한 방!],01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-21T11:45:06+09:00,True
4,30340227,월세,1000,49,23.14,3,⭕망원역세권 7평원룸 내부깔끔⭕,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-01-22T14:38:50+09:00,False


In [80]:
result_df[(result_df['deposit'] <= 10000) & (result_df['rent'] <= 100)]

,item_id,sales_title,deposit,rent,size_m2,floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
4,30340227,월세,1000,49,23.14,3,⭕망원역세권 7평원룸 내부깔끔⭕,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-01-22T14:38:50+09:00,False
5,30691402,월세,5000,90,33.06,4,🌈여성1인전용📢대출가능👑망원역6분🌈테라스💖,02,마포구 망원동,원룸,서울시 마포구 망원동,10,2022-02-18T12:12:34+09:00,False
6,30604487,전세,9300,0,15.00,2,망리단길 인접 풀옵션 원룸,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-02-21T02:39:16+09:00,True
15,30687981,월세,1000,85,49.59,3,⭕망원역세권 남향투룸 내부깔끔⭕,04,마포구 망원동,빌라,서울시 마포구 망원동,5,2022-02-18T09:50:36+09:00,False
16,30488407,월세,500,43,19.83,옥탑방,🤩이넓은테라스가다내꺼🌞,02,마포구 망원동,원룸,서울시 마포구 망원동,2,2022-02-16T17:10:20+09:00,False
20,30620775,월세,3000,100,52.89,3,⭕망원동 한강공원인근 투룸월세⭕,04,마포구 망원동,빌라,서울시 마포구 망원동,2,2022-02-14T10:25:58+09:00,False
26,30592602,월세,2000,55,16.53,3,🎉이쁜 인테리어 풀옵 원룸🎉,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-15T12:53:03+09:00,False
30,30522407,월세,10000,60,42.84,2,중기청80% 통베란다 넓고 전대가능한 투룸 월세,04,마포구 망원동,빌라,서울시 마포구 망원동,3,2022-02-07T14:39:54+09:00,False
34,30707009,월세,3000,100,59.50,2,⭕망원시장안 쓰리룸 월세 주거용사무실가능⭕,05,마포구 망원동,빌라,서울시 마포구 망원동,0,2022-02-19T13:02:03+09:00,False
36,30728052,월세,5000,50,46.28,2,*!* 망원시장 올수리된 투룸 전월세 *!^,04,마포구 망원동,빌라,서울시 마포구 망원동,5,2022-02-21T12:52:41+09:00,True


In [81]:
import geohash2

def oneroom(address) :
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸'
    response = requests.get(url)
    datas = response.json()['items'][0]
    lat, lng = datas['lat'], datas['lng']
    
    # precision이 커질수록 영역이 작아짐
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    response = requests.get(url)
    datas = response.json()['items']
    ids = [i['item_id'] for i in datas]

    url = 'https://apis.zigbang.com/v2/items/list'
    params = {
        'domain': "zigbang",
        "item_ids" : ids[:998],
        'withCoalition': 'true',
    }
    response = requests.post(url, params)
    
    columns = ['item_id','sales_title', 'deposit', 'rent', 'size_m2','floor','title',
           'room_type', 'address','service_type', 'address1', 'manage_cost', 'reg_date', 'is_new']
    result_df = pd.DataFrame(items, columns=columns)
    
    result_df = result_df[result_df['address'].str.contains(address)]
    result_df =result_df.reset_index(drop=True)

    return result_df

In [83]:
oneroom('망원동').head()

,item_id,sales_title,deposit,rent,size_m2,floor,title,room_type,address,service_type,address1,manage_cost,reg_date,is_new
0,30483493,전세,23000,0,18.99,6,"🦋합정역,신축1.5룸가성비짱,한강공원도보,보증보험OK",02,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-04T15:40:23+09:00,False
1,30608827,전세,23000,0,42.98,6,방1개 별도거실 주방 화장실 다용도실 분리형원룸 입니다,04,마포구 망원동,빌라,서울시 마포구 망원동,7,2022-02-12T16:27:27+09:00,False
2,30578522,월세,12000,10,12.21,3,💥LH 가능 깨끗한 방💥,01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-15T18:54:14+09:00,False
3,30726074,월세,12000,10,13.22,3,🎶 [LH 가능!] 🎶 [깔끔한 방!],01,마포구 망원동,원룸,서울시 마포구 망원동,5,2022-02-21T11:45:06+09:00,True
4,30340227,월세,1000,49,23.14,3,⭕망원역세권 7평원룸 내부깔끔⭕,01,마포구 망원동,원룸,서울시 마포구 망원동,3,2022-01-22T14:38:50+09:00,False
